In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,25106
2,Huelva,Confirmados PDIA 14 días,410
3,Huelva,Tasa PDIA 14 días,"79,89555118186955"
4,Huelva,Confirmados PDIA 7 días,245
5,Huelva,Total Confirmados,25305
6,Huelva,Curados,19556
7,Huelva,Fallecidos,347


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  25106.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6640.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3688 personas en los últimos 7 días 

Un positivo PCR cada 2085 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,25106.0,245.0,410.0,513170.0,47.742464,79.895551,48.0
Huelva-Costa,15251.0,97.0,183.0,289548.0,33.500490,63.201956,25.0
Condado-Campiña,7128.0,137.0,210.0,156231.0,87.690663,134.416345,18.0
Almonte,928.0,46.0,56.0,24507.0,187.701473,228.506141,12.0
Alosno,136.0,13.0,14.0,3933.0,330.536486,355.962370,8.0
Huelva (capital),6640.0,39.0,69.0,143837.0,27.114025,47.970967,7.0
Cartaya,981.0,15.0,25.0,20083.0,74.690036,124.483394,7.0
Sierra de Huelva-Andévalo Central,2310.0,7.0,11.0,67391.0,10.387144,16.322654,2.0
Valverde del Camino,425.0,6.0,9.0,12750.0,47.058824,70.588235,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puebla de Guzmán,108.0,6.0,13.0,3092.0,194.049159,420.439845,0.0
Villalba del Alcor,458.0,7.0,13.0,3366.0,207.961973,386.215092,0.0
Alosno,136.0,13.0,14.0,3933.0,330.536486,355.962370,8.0
San Bartolomé de la Torre,212.0,6.0,13.0,3761.0,159.532039,345.652752,0.0
Almonte,928.0,46.0,56.0,24507.0,187.701473,228.506141,12.0
Gibraleón,531.0,15.0,20.0,12737.0,117.767135,157.022847,1.0
Rociana del Condado,324.0,11.0,12.0,7939.0,138.556493,151.152538,1.0
Moguer,859.0,22.0,30.0,21867.0,100.608222,137.193031,-1.0
Condado-Campiña,7128.0,137.0,210.0,156231.0,87.690663,134.416345,18.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Ayamonte,1235.0,4.0,22.0,21104.0,18.953753,104.245641,2.0,0.181818
Palma del Condado (La),988.0,2.0,10.0,10801.0,18.516804,92.584020,0.0,0.200000
San Juan del Puerto,368.0,3.0,9.0,9411.0,31.877590,95.632770,0.0,0.333333
San Bartolomé de la Torre,212.0,6.0,13.0,3761.0,159.532039,345.652752,0.0,0.461538
Puebla de Guzmán,108.0,6.0,13.0,3092.0,194.049159,420.439845,0.0,0.461538
Escacena del Campo,141.0,1.0,2.0,2287.0,43.725404,87.450809,0.0,0.500000
Palos de la Frontera,462.0,5.0,10.0,11742.0,42.582184,85.164367,1.0,0.500000
Lepe,2009.0,8.0,16.0,27880.0,28.694405,57.388809,0.0,0.500000
Huelva-Costa,15251.0,97.0,183.0,289548.0,33.500490,63.201956,25.0,0.530055
